In [11]:
import pandas as pd
import numpy as np
import itertools

In [12]:
def draw_sudoku(board):
    print("+" + "---+"*9)
    for i, row in enumerate(board):
        print(("|" + " {}   {}   {} |"*3).format(*[x if x != 0 else "0" for x in row]))
        if i % 3 == 2:
            print("+" + "---+"*9)

In [13]:
df = pd.read_csv('example1.csv', header=None)
matrix = df.to_numpy()
d=[]
x = 0
for i in range(matrix.shape[0]):
    for j in range(matrix.shape[1]):
        if(matrix[i,j] > 0):
            d.append( [[i,j],matrix[i,j]])
            x+=1
draw_sudoku(matrix)

+---+---+---+---+---+---+---+---+---+
| 0   0   6 | 0   0   0 | 5   0   7 |
| 0   9   0 | 0   6   8 | 0   0   0 |
| 7   0   8 | 0   0   5 | 0   0   4 |
+---+---+---+---+---+---+---+---+---+
| 0   0   1 | 5   0   4 | 2   9   0 |
| 0   7   0 | 0   3   0 | 0   0   0 |
| 3   0   9 | 0   0   0 | 7   1   0 |
+---+---+---+---+---+---+---+---+---+
| 0   5   0 | 0   4   0 | 8   0   9 |
| 0   8   2 | 6   0   0 | 0   0   0 |
| 0   0   0 | 0   2   7 | 4   0   6 |
+---+---+---+---+---+---+---+---+---+


In [14]:
def fitness(m):
    side = m.shape[0]
    def repeated(st):
        st, c = np.unique(st, return_counts = True)
        if st[0] != 0: return np.count_nonzero(c > 1)
        return np.count_nonzero(c[1:] > 1)
    result = side**2
    for i in range(side):
        result -= repeated(m[i, :])
        result -= repeated(m[:, i])
        sqrt = np.sqrt(side)
        h, v = (i % sqrt)*sqrt, (i // sqrt)*sqrt
        result -= repeated(m[int(v): int(v + sqrt), int(h): int(h + sqrt)])
    return result - np.count_nonzero(m == 0)
fitness(matrix)

32

In [1]:
def geneticAlgorithm(m, n, g, pR, pM): #Starting sudoku, Population Size, # of generations, P(reproduction), p(Mutation), # of queens
    sideSize = m.shape[0]
    population = np.array(["""Sara's""" for i in range(n)])
    best, fitnesses = -1, [fitness(i) for i in population]        
    best = np.argmax(fitnesses)
    newPopulation, newFitnesses = np.zeros((sideSize, sideSize, n)), np.zeros(n)
    for i in range(g):
        if fitnesses[best] == sideSize**2:
            return population[best]
        for childIndex in range(n):
            #pOne = population[tournament(population, 2*nCols//3)]
            #pTwo = population[tournament(population, 2*nCols//3)]
            pOne = population[roulette(population)]
            pTwo = population[roulette(population)]
            parents = np.array([pOne, pTwo])
            newChild = simplePermutationCrossover(pR, parents)
            newChild = mutation(pM, newChild)
            newFitnesses[childIndex] = fitness(newChild)
            newPopulation[childIndex] = newChild
        betterBest = np.argmax(newFitnesses)
        if newFitnesses[betterBest] < fitnesses[best]: newPopulation[tournament(population, n//2)] = population[best]
        else: best = betterBest
        population, fitnesses = newPopulation, newFitnesses
    return population[best]

In [16]:
def tournamentSelection(fitnesses, k = 2, reversed = False):
    participants = np.random.choice(len(fitnesses), k, False)
    if reversed: return np.argmin(fitnesses[participants])
    return np.argmax(fitnesses[participants])

In [17]:
def rouletteSelection(fitnesses):
    if len(set(fitnesses)) == 1:
        return np.random.choice(len(fitnesses))
    norm = (fitnesses - np.min(fitnesses)) / (np.max(fitnesses) - np.min(fitnesses))
    probs = np.divide(norm, sum(norm))
    return np.random.choice(len(fitnesses), replace = False, p = probs)